### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = True

True

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [9]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [10]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several pros of using AutoGen in your new AI Agent project:

1. **Multi-Agent Collaboration**: AutoGen is designed for orchestrating multiple AI agents that can work together, which enhances problem-solving capabilities and allows for more sophisticated interactions.

2. **Enhanced LLM Inference**: By leveraging large language models effectively, AutoGen can improve the efficiency and accuracy of responses generated by agents, making it suitable for complex tasks.

3. **Built-in Code Execution**: AutoGen supports executing code through built-in executors, allowing agents to autonomously perform tasks that require computational capabilities, thus automating complex workflows.

4. **High Customization**: The framework offers fine-grained control, enabling developers to tailor agents specifically for their needs, enhancing the flexibility of implementations.

5. **Scalability**: AutoGen is built to scale well, which makes it suitable for applications that require handling large data sets or complex, dynamic environments.

6. **Open Source**: Being open-source, it encourages community contributions and modifications, allowing users to benefit from collective improvements and shared advancements.

In summary, AutoGen brings powerful multi-agent capabilities and flexibility to AI projects, enhancing automation and problem-solving efficiency in complex scenarios. 

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen in your AI Agent project:

1. **Dependency on Input Quality**: The effectiveness of AutoGen is heavily reliant on the quality of input data. Poor input can lead to undesirable or inaccurate outputs, requiring users to invest extra time in selecting and validating inputs carefully.

2. **Limited Customization**: While AutoGen can generate content quickly, it may lack the flexibility for deep customization needed in specific domains or tasks. This could result in outputs that may not fully meet the project's unique requirements.

3. **Potential for Bias**: If the training data for AutoGen contains biases, these biases can be reflected in the generated outputs. This could lead to ethical concerns or misrepresentation in the context of sensitive topics.

4. **Complexity in Fine-Tuning**: Fine-tuning the models for specific use cases can be complex and may require a deep understanding of both the tool and the underlying data structures, which can increase the learning curve for developers.

5. **Resource Intensive**: Generating high-quality content might require significant computational resources and can lead to increased costs, especially for large-scale applications.

6. **Risk of Output Repetition**: AutoGen may generate repetitive outputs or fail to introduce variability in content, which can diminish user engagement and reduce the effectiveness of AI agents in providing diverse and fresh content. 

These factors should be carefully considered before deciding to implement AutoGen in your project. 

TERMINATE



## Decision:

Based on the research provided by the team, I recommend using AutoGen for the project. The strengths of AutoGen, particularly its multi-agent collaboration capabilities, enhanced inference from large language models, and the ability to execute code built-in, present significant advantages for complex tasks. Additionally, its high customization potential and scalability position it well for future growth and adaptability.

While the cons, such as dependency on input quality and the potential for bias, are valid concerns, they can be mitigated with diligent data management and ethical guidelines. The overall benefits of improved problem-solving capabilities and automation capabilities support the decision to adopt AutoGen.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()